# Stochastic model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math


In [ ]:
#open the meteorological data
import pandas as pd
meteo_df = pd.read_csv('../Data/meteo_tetiaroa_resampled.csv')
print("Columns in meteo_df:", meteo_df.columns)
#remove nan in precip
meteo_df = meteo_df.dropna(subset=['precip', 'precip1', 'precip2', 'precip4', 'UM', 'TM', 'date', 'time'])
precip_data = meteo_df['precip'].values
precip1_data = meteo_df['precip1'].values
precip2_data = meteo_df['precip2'].values
precip4_data = meteo_df['precip4'].values
hum_data = meteo_df['UM'].values
temperature_data = meteo_df['TM'].values
dates_data = meteo_df['date'].values
dates_dt = pd.to_datetime(dates_data)
time_data = meteo_df['time'].values

In [ ]:
mean_rain = np.mean(precip_data)
max_rain = np.max(precip_data)
print("Mean Rain:", mean_rain)
print("Max Rain:", max_rain)

In [ ]:
plt.plot(dates_dt, hum_data)
plt.title("Cumsum of deltarpcip normalized")
plt.ylabel("Cumsum deltarpcip")
plt.xlabel("Date")
plt.show()

In [ ]:
Precmax = np.max(precip_data)
deltaprecip = 0.00001 * (25 + temperature_data ** 2) * (100 - hum_data)
# Cumsum of the deltaprecip normalized by Precmax, where value minimum is 0 and maximum is Precmax
H = np.zeros_like(precip_data)
H[0] = mean_rain*6
for i in range(1, len(precip_data)):
    H[i] = min(max_rain, max(0, H[i-1] + precip_data[i] - deltaprecip[i]))

In [ ]:
plt.plot(dates_dt, H)
plt.title("Cumsum of deltarpcip normalized")
plt.ylabel("Cumsum deltarpcip")
plt.xlabel("Date")
plt.show()

The following models will need the same rates and functions for events. Here, the temperature and rainfall dependencies only account for the competition in the larval stage.

In [ ]:
def allee(M, Ms):
    return (0.1 *M + 0.01 *Ms) / (1 + 0.1 *M + 0.01 * Ms)

def allee2(M, Ms):
    return 1 - np.exp(-0.05 *(M + 0.1 *Ms))

def release1(M, n):
    return n*M + 2000

def competition1(K0, Kh, precip):
    return 1/(K0 + Kh *  precip)

def competition2( R0, Rmax, K0, Kh, precip, temp, hum, k = 1):
    delta_rain = precip + k * (25+ temp**2) * (100 - hum)
    rain = max(0, min(R0 + delta_rain, Rmax))/Rmax
    return Kh * rain + K0

In [ ]:
#plot allee for Ms = 0 and M from 0 to 1000
M = np.linspace(0, 500, 100)
plt.plot(M, allee(M, 0), label="Holling type 2 effect")
plt.plot(M, allee2(M, 0), label="Exponential effect")
plt.title("Allee effect for Ms = 0")
plt.ylabel("Allee effect")
plt.xlabel("M")
plt.legend()
plt.show()

In [ ]:
#plot the competition through time
comp_history = []
c = 0.01
T_max = 720
for t in range(1, T_max):
    precip = np.interp(t, time_data, precip2_data)
    comp = competition1(1/c, 1/(c*30), precip)
    comp_history.append(comp)

plt.figure(figsize=(10, 6))
plt.plot(range(1, T_max), comp_history, label='Competition Coefficient over Time')
plt.xlabel('Time (days)')
plt.ylabel('Competition Coefficient')
plt.title('Competition Coefficient Dynamics')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#plot the competition through time
comp_history = []
c = 0.01
T_max = 720
for t in range(1, T_max):
    precip = np.interp(t, time_data, precip2_data)
    comp = competition1(1/c, 1/(c*30), precip)
    comp_history.append(comp)

fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot competition coefficient on left y-axis
ax1.plot(range(1, T_max), comp_history, label='Competition Coefficient over Time', color='blue')
ax1.set_xlabel('Time (days)')
ax1.set_ylabel('Competition Coefficient', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title('Competition Coefficient Dynamics')
ax1.grid()

# Create second y-axis for precipitation
ax2 = ax1.twinx()
precip_history = [np.interp(t, time_data, precip2_data) for t in range(1, T_max)]
ax2.plot(range(1, T_max), precip_history, label='Precipitation', color='red', alpha=0.7)
ax2.set_ylabel('Precipitation', color='red')
ax2.tick_params(axis='y', labelcolor='red')

fig.tight_layout()
plt.show()

In [ ]:
plt.plot(dates_dt, 1/((  1 /(c) * H / (max_rain) + 1/ (5 *c))))
plt.title("Cumsum of deltarpcip normalized")
plt.ylabel("Cumsum deltarpcip")
plt.xlabel("Date")
plt.show()

In [ ]:
init_egg = 10000  # Starting egg population
init_l = 1000 # Starting larval population
init_p = 1000 # Starting pupal population
init_f0 = 100 # Starting female population
init_ff = 1200 # Starting fertilised female pop
init_fs = 0 # Starting strerile female pop
init_male = 400 # Starting adult population
init_sterile = 0 # Starting sterile male population
comp_larvae = 0.0001 # Competition coefficient for larvae
n_rel = 0
n_egg = 64

## Full mosquito life cycle

In [ ]:
# Parameters
birth = 10/64  # Birth rate per adult
mu = 0.51    # rate at which egg becomes a female
egg_death = 0.046    # Death rate per egg
larva_death = 0.05   # Death rate per larva
pupa_death = 0.05   # Death rate per pupa
female_death = 0.046  # Death rate per adult
male_death = 0.139    # Death rate per adult
sterile_death = male_death*1.2   # Death rate per sterile
c = 0.01 # Competition coefficient for larvae
transi_el = 0.79
transi_lp = 0.125
transi_pa = 0.125
t=0.0

In [ ]:
T_max = time_data[-1]
pop = [init_egg, init_l, init_p, init_f0, init_ff, init_fs, init_male, init_sterile]
time_history = [t]
population_history = [pop]

In [ ]:
while t < 576.97:
    # Step 1: Calculate total rate
    T = np.interp(t, time_data, temperature_data) + 273.15
    precip = np.interp(t, time_data, precip2_data)
    #T = temperature_const(t) # Uncomment this line to use constant temperature
    #water = np.interp(t, time_data, H)

    comp = competition1(1/c, 1/(c * 50), precip)
    #comp = 1 /((1 /(c) * water / (max_rain) + 1/(5 * c)))
    
    if pop[6] > 0:
        prob_M = pop[6]/(pop[6] + pop[7])
        prob_Ms = pop[7]/(pop[6] + pop[7])
    else:
        prob_M = 0
        prob_Ms = 0
        
    total_rate = (egg_death * pop[0]+
        (larva_death + comp * pop[1]) * pop[1] +
        pupa_death * pop[2] +
        female_death * (pop[3] + pop[4] + pop[5]) +
        male_death * pop[6] +
        sterile_death * pop[7] +  
        birth * pop[4] +
        transi_el * pop[0] +
        transi_lp * pop[1] +
        transi_pa * pop[2] +
        allee(pop[6], pop[7]) * prob_M * pop[3] +
        allee(pop[6], pop[7]) * prob_Ms * pop[3])
    
    if total_rate == 0:
        print("No life :(")
        break

    # Step 2: Draw time to next event
    time_to_next_event = np.random.exponential(1 / total_rate)
    t += time_to_next_event

    # Step 4: Determine the event
    p = np.random.rand() * total_rate
    s = (egg_death * pop[0]) / total_rate
    if p < (egg_death * pop[0]):
        pop[0] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1]) :
        pop[1] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2]):
        pop[2] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * pop[3]):
        pop[3] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4])):
        pop[4] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5])):
        pop[5] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6]):
        pop[6] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7]):
        pop[7] -= 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4]):
        pop[0] += n_egg
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0]):
        pop[0] -= 1
        pop[1] += 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1]):
        pop[1] -= 1
        pop[2] += 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + 0.5 * transi_pa * pop[2]):
        pop[2] -= 1
        pop[6] += 1
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2]):
        pop[2] -= 1
        pop[3] += 1
    
    elif p < (egg_death * pop[0] + (larva_death + comp * pop[1]) * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2] + allee(pop[6], pop[7]) * prob_M * pop[3]):
        pop[3] -= 1
        pop[4] += 1
    #elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2] + allee(pop[6], pop[7]) * prob_M * pop[3] + allee(pop[6], pop[7]) * prob_Ms * pop[3]):
    #elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2] + allee(pop[6], pop[7]) * prob_M * pop[3] + allee(pop[6], pop[7]) * prob_Ms * pop[3] + ):
    else :
        pop[3] -= 1
        pop[5] += 1
    
    # Step 3: Sterile insect release
    is_release_day = int(t) % 7 == 0
    is_new_day     = int(time_history[-1]) != int(t)
    enough_females = pop[4] + pop[5] > 30
    after_start    = t > 250
    releases_left  = n_rel < 20

    if is_release_day and is_new_day and enough_females and after_start and releases_left:
        nb = max(10000,release1(pop[6], 100))
        pop[7] += nb
        n_rel += 1
        print(f"Release {n_rel} at t={t:.2f}: {nb} sterile males")

    # Step 4: Record history
    time_history.append(t)
    population_history.append((pop[0], pop[1], pop[2], pop[3], pop[4], pop[5], pop[6], pop[7]))
    if sum(pop) == 0 :
        print(f"No life at time {t:.2f}:(")
        break
    
    if min(pop) < 0:
        print(f"Negative population at time {t:.2f}:(")
        break


In [ ]:
len(time_history)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
temp = np.interp(np.array(time_history), time_data, temperature_data)

# Sample every 10th point
step = 10
time_sampled = time_history[::step]
pop_sampled = population_history[::step]

# Plot results for male and female population
axs[0, 0].step(time_sampled, [pop[3] for pop in pop_sampled], where="post", label='F0', color = 'blue')
axs[0, 0].step(time_sampled, [pop[4] for pop in pop_sampled], where="post", label='Ff', color = 'red')
axs[0, 0].step(time_sampled, [pop[5] for pop in pop_sampled], where="post", label='Fs', color = 'grey')
axs[0, 0].set_xlabel('Time')
axs[0, 0].set_ylabel('Population Size')
axs[0, 0].set_ylim(bottom=0)
axs[0, 0].set_title('Female Population')
axs[0, 0].legend()

# Plot results for sterile male population
axs[0, 1].step(time_sampled, [pop[7] for pop in pop_sampled], where="post", label='Sterile Male', color='orange')
axs[0, 1].set_xlabel('Time')
axs[0, 1].set_ylabel('Sterile Male Population Size')
axs[0, 1].set_ylim(bottom=0)
axs[0, 1].set_title('Sterile Male Population')
axs[0, 1].legend()

# Plot results for egg population
axs[1, 0].step(time_sampled, [pop[0] for pop in pop_sampled], where="post", label='Egg', color='green')
axs[1, 0].set_xlabel('Time')
axs[1, 0].set_ylabel('Population Size')
axs[1, 0].set_ylim(bottom=0)
axs[1, 0].set_title('Aquatic Population')
axs[1, 0].legend()

# Create a secondary y-axis for larva and pupa
ax2 = axs[1, 0].twinx()
ax2.step(time_sampled, [pop[1] for pop in pop_sampled], where="post", label='Larva', color='blue')
ax2.step(time_sampled, [pop[2] for pop in pop_sampled], where="post", label='Pupa', color='purple')
ax2.set_ylabel('Larva and Pupa Population Size')
ax2.set_ylim(bottom=0)
ax2.legend(loc='upper right')

axs[1, 1].step(time_sampled, [pop[3] + pop[4] + pop[5] for pop in pop_sampled], where="post", label='F0+Ff+Fs', color='purple')
axs[1, 1].step(time_sampled, [pop[6] for pop in pop_sampled], where="post", label='M', color='black')
axs[1, 1].set_xlabel('Time')
axs[1, 1].set_ylabel('Population Size')
axs[1, 1].set_ylim(bottom=0)
axs[1, 1].set_title('F0 and fertile female Population')
axs[1, 1].legend()

plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
temp = np.interp(np.array(time_history), time_data, temperature_data)

# Sample every 10th point
step = 10
time_sampled = time_history[::step]
pop_sampled = population_history[::step]

# Plot results for male and female population
axs[0, 0].step(time_sampled, [pop[3] for pop in pop_sampled], where="post", label='F0', color = 'blue')
axs[0, 0].step(time_sampled, [pop[4] for pop in pop_sampled], where="post", label='Ff', color = 'red')
axs[0, 0].step(time_sampled, [pop[5] for pop in pop_sampled], where="post", label='Fs', color = 'grey')
axs[0, 0].set_xlabel('Time')
axs[0, 0].set_ylabel('Population Size')
axs[0, 0].set_ylim(bottom=0)
axs[0, 0].set_title('Female Population')
axs[0, 0].legend()

# Plot results for sterile male population
axs[0, 1].step(time_sampled, [pop[7] for pop in pop_sampled], where="post", label='Sterile Male', color='orange')
axs[0, 1].set_xlabel('Time')
axs[0, 1].set_ylabel('Sterile Male Population Size')
axs[0, 1].set_ylim(bottom=0)
axs[0, 1].set_title('Sterile Male Population')
axs[0, 1].legend()

# Plot results for egg population
axs[1, 0].step(time_sampled, [pop[1] for pop in pop_sampled], where="post", label='Larva', color='green')
axs[1, 0].set_xlabel('Time')
axs[1, 0].set_ylabel('Population Size')
axs[1, 0].set_ylim(bottom=0)
axs[1, 0].set_title('Larva Population')
axs[1, 0].legend()

axs[1, 1].step(time_sampled, [pop[3] + pop[4] + pop[5] for pop in pop_sampled], where="post", label='F0+Ff+Fs', color='purple')
axs[1, 1].step(time_sampled, [pop[6] for pop in pop_sampled], where="post", label='M', color='black')
axs[1, 1].set_xlabel('Time')
axs[1, 1].set_ylabel('Population Size')
axs[1, 1].set_ylim(bottom=0)
axs[1, 1].set_title('F0 and fertile female Population')
axs[1, 1].legend()

plt.tight_layout()
plt.show()


## Only Larva

Simplified aquatic state with only larva stage with updated rates.

In [ ]:
#modified rates for the model

## Only adults

Simplest model, where only adult mosquitoes are modelled. It depends on the previous rates and functions, with a square root of the female populatoin in birth.

$\beta_A(A) = \displaystyle \frac{-(\delta_L + \tau_L) + \sqrt{(\delta_L + \tau_L)^2 + 4c \beta(A) A}}{2c}$

In [ ]:
pop3 = [init_f0 + init_ff, init_male, init_sterile]
t = 0.0
time3_history = [t]
population3_history = [pop3]

In [ ]:
transiL_mod = larva_death ** 2 / (pupa_death + transi_pa)

while t < T_max/4:
    # Step 1: Calculate total rate
    T = np.interp(t, time_data, temperature_data) + 273.15
    precip = np.interp(t, time_data, precip2_data)
    #T = temperature_const(t) # Uncomment this line to use constant temperature
    #water = np.interp(t, time_data, H)

    comp = competition1(1/c, 1/(c * 50), precip)
    #comp = 1 /((1 /(c) * water / (max_rain) + 1/(5 * c)))
    
    if pop3[1] > 0:
        prob_M = pop3[1]/(pop3[1] + pop3[2])
    else:
        prob_M = 0

    birth_mod = birth * prob_M * allee(pop3[1], pop3[2]) * transi_el / (transi_el + egg_death)
    birth_rate = ( -(larva_death + transiL_mod) + np.sqrt((larva_death + transiL_mod)**2 + 4 * comp * birth_mod) )/ (2 * comp)
    
    total_rate = (female_death * pop3[0]+
                  male_death * pop3[1]+
                  male_death * 1.2 * pop3[2]+
                  birth_rate * pop3[0]
    )

    if total_rate == 0:
        print("No life :(")
        break
    
    # Step 2: Draw time to next event
    time_to_next_event = np.random.exponential(1 / total_rate)
    t += time_to_next_event

    # Step 4: Determine the event
    p = np.random.rand() * total_rate
    if p < (female_death * pop3[0]):
        pop3[0] -= 1
    elif p < (female_death * pop3[0] + male_death * pop3[1]):
        pop3[1] -= 1
    elif p < (female_death * pop3[0] + male_death * pop3[1] + male_death * 1.2 * pop3[2]):
        pop3[2] -= 1
    else:
        pop3[0] += n_egg/2
        pop3[1] += n_egg/2


    # Step 3: Sterile insect release
    is_release_day = int(t) % 7 == 0
    is_new_day     = int(time_history[-1]) != int(t)
    enough_females = pop[4] + pop[5] > 30
    after_start    = t > 250
    releases_left  = n_rel < 0

    if is_release_day and is_new_day and enough_females and after_start and releases_left:
        nb = max(10000,release1(pop[6], 100))
        pop[7] += nb
        n_rel += 1
        print(f"Release {n_rel} at t={t:.2f}: {nb} sterile males")


    # Step 5: Record history
    time3_history.append(t)
    population3_history.append((pop3[0], pop3[1], pop3[2]))
    if sum(pop3) == 0 or pop3[1] + pop3[2] == 0:
        print("No life :(")
        break
    
    if min(pop3) < 0:
        print("Negative population :(")
        break


In [ ]:
# Plot male and female, and a plot to the right with sterile male
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

step = 1000
time3_sampled = time3_history[::step]
pop3_sampled = population3_history[::step]

# Plot results for male and female population
axs[0].step(time3_sampled, [pop[0] for pop in pop3_sampled], where="post", label='F', color = 'blue')
axs[0].step(time3_sampled, [pop[1] for pop in pop3_sampled], where="post", label='M', color = 'red')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Population Size')
axs[0].set_ylim(bottom=0)
axs[0].set_title('Female and Male Population')
axs[0].legend()

# Plot results for sterile male population
axs[1].step(time3_sampled, [pop[2] for pop in pop3_sampled], where="post", label='Sterile Male', color='orange')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Sterile Male Population Size')
axs[1].set_ylim(bottom=0)
axs[1].set_title('Sterile Male Population')
axs[1].legend()